# 2D / 3D Mapping

 ## Boreholes 2D mapping from files

### imports

In [1]:
import matplotlib.pyplot as plt
#from striplog import Component, Decor, Legend
#from core.omf import Borehole3D, striplog_legend_to_omf_legend 

import geopandas as gpd, pandas as pd
from shapely import wkt
import folium as fm
from folium import plugins
from utils.io import boreholes_from_files, striplog_from_text, export_gdf, read_gdf_file
from os import remove
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

### Data from a shapefile

In [2]:
file = ["/CF_data/gis/Pilote_v7.0.shp","/CF_data/gis/Pilote_v8.0.shp"]
shp = gpd.read_file(ROOT_DIR+file[1])
shp

,Id,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
0,0,1,1,111,2,1,1,0,1,0,OUI,POINT (152894.891 122592.127)
1,0,1,1,120,2,1,1,0,1,0,OUI,POINT (152881.447 122588.102)
2,0,2,1,217,3,1,1,1,1,0,OUI,POINT (152885.995 122587.127)
3,0,1,1,116,2,1,1,0,1,0,OUI,POINT (152883.768 122589.754)
4,0,2,1,214,2,1,1,1,1,0,OUI,POINT (152888.128 122588.436)
...,...,...,...,...,...,...,...,...,...,...,...,...
83,0,6,0,50,0,0,0,0,0,0,OUI,POINT (152893.126 122602.456)
84,0,6,0,52,0,0,0,0,0,0,OUI,POINT (152889.301 122592.517)
85,0,6,0,51,0,0,0,0,0,0,OUI,POINT (152889.534 122591.686)
86,0,6,0,53,0,0,0,0,0,0,OUI,POINT (152889.786 122588.802)


**Rmq** : ici à quoi correspondent :

    - ID (tout est à 0!), 
    - Ref
    - Type

In [3]:
if 'Id' in shp.columns:
    shp=shp.drop(['Id'], axis=1)
    #shp.index.name='Id'
shp

,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
0,1,1,111,2,1,1,0,1,0,OUI,POINT (152894.891 122592.127)
1,1,1,120,2,1,1,0,1,0,OUI,POINT (152881.447 122588.102)
2,2,1,217,3,1,1,1,1,0,OUI,POINT (152885.995 122587.127)
3,1,1,116,2,1,1,0,1,0,OUI,POINT (152883.768 122589.754)
4,2,1,214,2,1,1,1,1,0,OUI,POINT (152888.128 122588.436)
...,...,...,...,...,...,...,...,...,...,...,...
83,6,0,50,0,0,0,0,0,0,OUI,POINT (152893.126 122602.456)
84,6,0,52,0,0,0,0,0,0,OUI,POINT (152889.301 122592.517)
85,6,0,51,0,0,0,0,0,0,OUI,POINT (152889.534 122591.686)
86,6,0,53,0,0,0,0,0,0,OUI,POINT (152889.786 122588.802)


In [4]:
print("nombre de données :", len(shp))
print("nombre de forages (diam > 0 ??) :", len(shp.query('diam>0')))
#shp.loc[41]

nombre de données : 88
nombre de forages (diam > 0 ??) : 60


### Save location data in a Geopackage / GeoJson / csv

In [5]:
export_gdf(shp, 31370, 'tmp_files/test_loc.csv')

tmp_files/test_loc.csv has been saved !


In [6]:
cat 'tmp_files/test_loc.csv'

X,Y,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
152894.89100002544,122592.12713904586,1,1,111,2,1,1,0,1,0,OUI,POINT (152894.8910000254 122592.1271390459)
152881.4470000254,122588.10213904455,1,1,120,2,1,1,0,1,0,OUI,POINT (152881.4470000254 122588.1021390446)
152885.99500002537,122587.12713904679,2,1,217,3,1,1,1,1,0,OUI,POINT (152885.9950000254 122587.1271390468)
152883.7680000254,122589.75413904712,1,1,116,2,1,1,0,1,0,OUI,POINT (152883.7680000254 122589.7541390471)
152888.1280000254,122588.43613904715,2,1,214,2,1,1,1,1,0,OUI,POINT (152888.1280000254 122588.4361390471)
152886.2260000254,122591.65513904672,1,1,113,2,1,1,0,1,0,OUI,POINT (152886.2260000254 122591.6551390467)
152880.9800000254,122583.69313904643,2,1,223,3,1,1,1,1,1,OUI,POINT (152880.9800000254 122583.6931390464)
152883.65800002537,122585.62313904893,2,1,221,2,1,1,1,1,1,OUI,POINT (152883.6580000254 122585.6231390489)
152889.1520000254,122587.1761390483,1,1,118,2,1,1,0,1,0,OUI,POINT (152889.15200002

### 2D Map

-idée à explorer : fonction pour générer une carte des forages présents dans un projet

In [7]:
#Change CRS EPSG 31370 (Lambert 72) into EPSG 4326 (WGS 84)
shp = shp.to_crs(epsg=4326)
center=[shp.geometry.y.mean(), shp.geometry.x.mean()]

# Use a satellite map
tile_name = ["Satellite"]
tile_attr = ["Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"]
tile_url = ["https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"]

BH_map=fm.Map(location=center, tiles='OpenStreetMap', zoom_start=19, max_zoom=25, control_scale=True)

ch1 = fm.FeatureGroup(name='Boreholes')

for idx, row in shp.iterrows():
    fm.CircleMarker([row.geometry.y, row.geometry.x], popup=str(idx) , radius=0.1,
                     color='red', fill_color='red',
                    opacity=0.6
                    ).add_to(ch1)
    
#fm.GeoJson('BH_loc.json').add_to(fm_map)

mini_map = plugins.MiniMap(toggle_display=True, zoom_level_offset=-6)

#adding features to the base_map
fm.TileLayer(name=tile_name[0], tiles=tile_url[0], max_zoom=25, attr=tile_attr[0], control=True).add_to(BH_map)
ch1.add_to(BH_map)
fm.LayerControl().add_to(BH_map)
BH_map.add_child(mini_map)

#save in a file
BH_map.save('tmp_files/BH_location.html')
BH_map

### Reading location data from a csv, Geo(package/Json) file

In [8]:
file_dir=ROOT_DIR+'/playground/TFE_test/tmp_files/'

In [9]:
print(file_dir)

/home/yanathan/Projects/GSDMA/playground/TFE_test/tmp_files/


In [10]:
bh_loc=read_gdf_file('test_loc.csv', 31370, 4326, file_dir)
bh_loc.head(2)

,X,Y,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
0,4.409477,50.414208,1,1,111,2,1,1,0,1,0,OUI,POINT (4.40948 50.41421)
1,4.409288,50.414172,1,1,120,2,1,1,0,1,0,OUI,POINT (4.40929 50.41417)


**Rmq** : la coordonnée Z est manquante dans les données de base !

### Create a boreholes positions list

In [11]:
x_list=bh_loc['X'].to_list()
y_list=bh_loc['Y'].to_list()
len(x_list)

88

## Borehole 3D mapping

### Build Borehole3D objects

Puisqu'on ne dispose que de la localisation des forages (pas d'infos sur la géométrie et le "contenu" des forages), l'idée est de créer **une géométrie et un contenu par défaut** pour chaque forage, **tout en respectant les positions**, puis les enregistrer dans la BD afin de pouvoir les afficher tous dans l'environnement 3D. La mise à jour des données manquantes se fera plus tard.

In [12]:
remove('tmp_files/tfe_orm_db.db')

Create the db engine and the database structure

In [13]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)
Base.metadata.create_all(engine)

2021-02-01 14:46:43,527 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-01 14:46:43,527 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 14:46:43,529 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-01 14:46:43,530 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 14:46:43,532 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2021-02-01 14:46:43,532 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 14:46:43,534 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2021-02-01 14:46:43,534 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 14:46:43,535 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2021-02-01 14:46:43,536 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 14:46:43,536 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2021-02-01 14:46:43,537 INFO sqlalchemy.engine.base.Engine ()
2021-02-01

creation of the Boreholes from las, csv files

In [14]:
lexicon = Lexicon.default() #to do : create our own lexicon

In [15]:
borehole_dict = {'F1':ROOT_DIR+'/data/test.las'}#, 'F2':ROOT_DIR+'/data/test.csv'} # boreholes to insert into the db

In [16]:
boreholes, components= boreholes_from_files(borehole_dict, x=x_list, y=y_list, verbose=0)

Create the session and the project

In [17]:
Session = sessionmaker(bind=engine)
session = Session()

p = Project(session)
p.add_components(components)

2021-02-01 14:46:44,133 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-01 14:46:44,136 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-01 14:46:44,137 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 14:46:44,140 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-02-01 14:46:44,140 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2021-02-01 14:46:44,141 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-01 14:46:44,145 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-01 14:46:44,145 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-01 14:46:44,145 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [18]:
for bh in boreholes:
    p.add_borehole(bh)

2021-02-01 14:46:44,156 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-02-01 14:46:44,157 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 4.409476785130411, 50.41420841912866), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 4.409476785130411, 50.41420841912866), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 4.409476785130411, 50.41420841912866), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 4.409476785130411, 50.41420841912866), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 4.409476785130411, 50.41420841912866), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 4.409476785130411, 50.41420841912866), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 4.409476785130411, 50.41420841912866), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 4.409476785130411, 50.41420841912866)  ... displaying 10 of 50 total bound p

2021-02-01 14:46:44,218 INFO sqlalchemy.engine.base.Engine (13,)
2021-02-01 14:46:44,222 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:46:44,223 INFO sqlalchemy.engine.base.Engine (14,)
2021-02-01 14:46:44,224 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:46:44,224 INFO sqlalchemy.engine.base.Engine (15,)
2021-02-01 14:46:44,228 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-01 14:46:44,290 INFO sqlalchemy.engine.base.Engine (34,)
2021-02-01 14:46:44,291 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:46:44,291 INFO sqlalchemy.engine.base.Engine (35,)
2021-02-01 14:46:44,296 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:46:44,297 INFO sqlalchemy.engine.base.Engine (36,)
2021-02-01 14:46:44,297 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2021-02-01 14:46:44,405 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2021-02-01 14:46:44,406 INFO sqlalchemy.engine.base.Engine ()
2021-02-01 14:46:44,408 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-02-01 14:46:44,409 INFO sqlalchemy.engine.base.Engine ('F2',)
2021-02-01 14:46:44,411 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:46:44,412 INFO sql

2021-02-01 14:46:44,468 INFO sqlalchemy.engine.base.Engine (69,)
2021-02-01 14:46:44,470 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:46:44,471 INFO sqlalchemy.engine.base.Engine (70,)
2021-02-01 14:46:44,472 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:46:44,473 INFO sqlalchemy.engine.base.Engine (71,)
2021-02-01 14:46:44,478 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

...

2021-02-01 14:47:04,384 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,385 INFO sqlalchemy.engine.base.Engine (4317,)
2021-02-01 14:47:04,388 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,390 INFO sqlalchemy.engine.base.Engine (4318,)
2021-02-01 14:47:04,392 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Posi

2021-02-01 14:47:04,450 INFO sqlalchemy.engine.base.Engine (4337,)
2021-02-01 14:47:04,453 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,454 INFO sqlalchemy.engine.base.Engine (4338,)
2021-02-01 14:47:04,455 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,456 INFO sqlalchemy.engine.base.Engine (4339,)
2021-02-01 14:47:04,460 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "

2021-02-01 14:47:04,574 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,574 INFO sqlalchemy.engine.base.Engine (4351,)
2021-02-01 14:47:04,577 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,578 INFO sqlalchemy.engine.base.Engine (4352,)
2021-02-01 14:47:04,579 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Posi

2021-02-01 14:47:04,643 INFO sqlalchemy.engine.base.Engine (4371,)
2021-02-01 14:47:04,648 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,649 INFO sqlalchemy.engine.base.Engine (4372,)
2021-02-01 14:47:04,650 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,652 INFO sqlalchemy.engine.base.Engine (4373,)
2021-02-01 14:47:04,655 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "

2021-02-01 14:47:04,713 INFO sqlalchemy.engine.base.Engine (4392,)
2021-02-01 14:47:04,714 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,715 INFO sqlalchemy.engine.base.Engine (4393,)
2021-02-01 14:47:04,720 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-02-01 14:47:04,721 INFO sqlalchemy.engine.base.Engine (4394,)
2021-02-01 14:47:04,722 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "

In [19]:
p.commit()

2021-02-01 14:47:04,795 INFO sqlalchemy.engine.base.Engine COMMIT


In [20]:
p.boreholes[0].intervals[0].description

2021-02-01 14:47:04,804 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-02-01 14:47:04,805 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-02-01 14:47:04,806 INFO sqlalchemy.engine.base.Engine ('F1',)
2021-02-01 14:47:04,807 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-02-01 14:47:04,808 INFO sqlalchemy.engine.base.Engine ('F1',)


'Anhydrite'

In [21]:
session.close()

2021-02-01 14:47:04,815 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [22]:
p.plot3d(x3d=True, radius=0.5)